## Create motion-based datasets from a Labelbox-style or COCO-style dataset

Types of datasets:
- Grids of crops of individual tracklets (grid-crops)
- On last image of tracklets, plot the past trajectories (traj-line-plots)
- On black background, plot all crops of the trajectory (traj-crop-plots)

In [10]:
# Global settings and datasets

# Labelbox dataset:
# labelbox_json = "/media/data/warp_data/wrsi-datasets/Labelbox-Export-WHOI-RSI-USVI-Fish-detect-and-track - 6_11_2024.ndjson"
# image_root_dir = "/media/data/warp_data/wrsi-datasets/whoi-rsi-fish-detection-yolo-dataset/images"

# COCO dataset:
coco_json = "/media/data/warp_data/marine_detection/imerit/whoi-rsi-fish-detection-datasets-22122023/coco.json"
image_root_dir = "/media/data/warp_data/marine_detection/imerit/whoi-rsi-fish-detection-datasets-22122023/"

output_root_dir = "/media/data/warp_data/wrsi-datasets/whoi-rsi-fish-motion-dataset"

In [2]:
# Setup Labelbox
import labelbox as lb
import labelbox.types as lb_types
import uuid
import base64
import requests

LABELBOX_API_KEY = "labelbox_api_key.txt"
ONTOLOGY_ID = "clqo6bd8v0jc407ybc1r9ehlb"
PROJECT_ID = 'clqoh3ylw1o8s070hd6ch5z7o' # WHOI RSI USVI Fish Track and Detect
DATASET_ID = "clqh7v7qi001r07886j6aws7i"

# Setup client
with open(LABELBOX_API_KEY,"r") as f:
    API_KEY = f.read().strip()
client = lb.Client(api_key=API_KEY)

ontology = client.get_ontology(ONTOLOGY_ID)
project = client.get_project(PROJECT_ID)
dataset = client.get_dataset(DATASET_ID)

In [19]:
# Imports
import cv2
import os
from tqdm import tqdm
import glob
from pathlib import Path
import json
import numpy as np

# Local imports
import labelbox_fish_utils as lbf

In [23]:
def crop_image(image_path, annotation, square_crop=True, resize=(244,244)):
    img = cv2.imread(image_path)
    x, y, w, h = annotation

    if square_crop:
        m = max(w,h)
        cx = int(x + w/2)
        cy = int(y + h/2)
        crop = img[int(cy-h/2):int(cy+h/2), int(cx-w/2):int(cx+w/2), :]
    else:
        crop = img[y:y+h, x:x+w, :]

    if resize:
        crop = cv2.resize(crop, resize)
    return crop

def generate_motion_dataset_from_labelbox(json_path, image_root_dir, output_root_dir):
    # Iterate through every tracklet
    pass

def generate_motion_dataset_from_coco(coco_json_path, image_root_dir, output_root_dir):

    # Parse COCO json
    with open(coco_json_path, "r") as f:
        coco = json.load(f)
    
    # Iterate through every object
    for object_track in tqdm(coco["object_tracks"]):
        bbox_id_list = object_track["bbox_id_list"]

        bboxes = []
        image_paths = []
        crops = []
        
        # Get bounding boxes and images
        for bbox_id in bbox_id_list:
            
            # Get bbox
            annotation = coco["annotations"][bbox_id]
            bbox = annotation["bbox"]

            # Get image
            image_id = annotation["image_id"]
            image = coco["images"][image_id]
            image_path = os.path.join(image_root_dir, image["file_name"])

            # Crop image
            crop = crop_image(image_path, bbox, resize=(244,244))

            bboxes.append(bbox)
            crops.append(crop)
            image_paths.append(image_path)

        # Make an image grid
        img_grid = np.hstack(crops)

        # Save stacked image
        video_path = coco["video_sequences"][object_track["video_seq_id"]]["file_name"]
        output_dir_path = os.path.join(output_root_dir, video_path)
        
        os.makedirs(output_dir_path, exist_ok=True)
        
        output_path = os.path.join(output_dir_path, f"{object_track['id']}.png")
        cv2.imwrite(output_path, img_grid)

In [24]:
generate_motion_dataset_from_coco(coco_json, image_root_dir, output_root_dir)

 14%|██████████████                                                                                    | 2260/15802 [06:36<39:33,  5.70it/s]


KeyboardInterrupt: 